# 遊びレコメンドモデル

------------------------------------------------

## 長期トレンドモデル

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

mm = preprocessing.MinMaxScaler()

filename = "./data/hangout.csv"
data = pd.read_csv(filename, index_col=0)
data.iloc[:,:] = mm.fit_transform(data)

In [53]:
for i in range(30):
    rand = np.random.randint(0, len(data))
    print(data.index[rand])

魚市場
ハイキング
ゲームセンター
イルミネーション
イルミネーション
陶芸・ガラス・手作り体験
占い
天体観測
占い
ショッピング
コンセプトバー
日帰り温泉
アニマルカフェ
陶芸・ガラス・手作り体験
魚市場
料理
フルーツ狩り
脱出ゲーム
ゲームセンター
ビリヤード
カラオケ
花火大会
イルミネーション
音楽教室・体験
アニマルカフェ
ビリヤード
エステ・マッサージ
天体観測
宅飲み
日帰り温泉


In [54]:
user = data.loc[["ショッピング","魚市場","日帰り温泉","トレッキング","ゲームセンター"]]
user

,アゴン(競争を伴う遊び),アレア(運やかけを伴う遊び),ミミクリ(真似・模倣を伴う遊び),イリンクス(目眩やスリルを伴う遊び)
ショッピング,0.0,0.0,0.4,0.0
魚市場,0.0,0.0,0.0,0.3
日帰り温泉,0.0,0.0,0.0,0.3
トレッキング,0.0,0.0,0.0,0.8
ゲームセンター,0.1,0.7,0.0,0.1


In [55]:
user_lterm = user.mean(axis=0)

In [56]:
user_lterm

アゴン(競争を伴う遊び)          0.02
アレア(運やかけを伴う遊び)        0.14
ミミクリ(真似・模倣を伴う遊び)      0.08
イリンクス(目眩やスリルを伴う遊び)    0.30
dtype: float64

In [57]:
result = np.linalg.norm(data - user_lterm, axis=1)
result

array([0.72965745, 0.25768197, 0.33823069, 0.22449944, 0.43174066,
       0.71860977, 0.94042544, 0.34117444, 1.0892199 , 0.60530984,
       0.93509358, 0.76052613, 0.28      , 1.00119928, 0.77097341,
       0.22      , 0.70313583, 0.88453378, 0.45431267, 0.42473521,
       0.43405069, 0.34117444, 0.75657121, 0.79271685, 0.19078784,
       0.16248077, 0.25768197, 0.14282857, 0.16248077, 0.27276363,
       0.16248077, 0.16248077, 0.16248077, 1.12623266, 0.91454907,
       0.25768197, 0.36386811, 0.61676576, 0.76052613, 0.63592452,
       0.21071308, 0.57480431, 0.46086874, 0.26153394, 0.64373908,
       0.64373908, 0.25768197, 0.34117444, 0.52573758, 0.71161787,
       0.99418308])

In [58]:
rank = np.argsort(result)
rank

array([27, 25, 32, 31, 30, 28, 24, 40, 15,  3,  1, 35, 26, 46, 43, 29, 12,
        2, 47,  7, 21, 36, 19,  4, 20, 18, 42, 48, 41,  9, 37, 39, 44, 45,
       16, 49,  5,  0, 22, 38, 11, 14, 23, 17, 34, 10,  6, 50, 13,  8, 33])

In [59]:
print("おすすめ度No.1 >> {}".format(data.iloc[rank[0]].name))
print("おすすめ度No.2 >> {}".format(data.iloc[rank[1]].name))
print("おすすめ度No.3 >> {}".format(data.iloc[rank[2]].name))
print("おすすめ度No.4 >> {}".format(data.iloc[rank[3]].name))
print("おすすめ度No.5 >> {}".format(data.iloc[rank[4]].name))

おすすめ度No.1 >> 日帰りツアー
おすすめ度No.2 >> 日帰り温泉
おすすめ度No.3 >> 酒造見学
おすすめ度No.4 >> 魚市場
おすすめ度No.5 >> 花火大会


----------------------------------------------------------------

## 短期トレンドモデル

Question1: 非日常を体験したいですか?  
0:したくない  …  4:したい

Question2: ゆったりのんびり遊びたいですか? エキサイティングな遊びをしたいですか?  
0:ゆったりのんびり  …  4:エキサイティング

In [120]:
q1 = (3 * 25) / 100
q2 = (3 * 25) / 100

アゴンの上昇 = q1が低 + q2が高  
アレアの上昇 = q1が低 + q2が低  
ミミクリの上昇 = q1が高 + q2が低  
イリンクスの上昇 = q1が高 + q2が高

In [121]:
alpha = 0.75

agon    = alpha * user_lterm[0] + (1 - alpha) * 0.5 * ((1-q1) + q2)
alea    = alpha * user_lterm[1] + (1 - alpha) * 0.5 * ((1-q1) + (1-q2))
mimicry = alpha * user_lterm[2] + (1 - alpha) * 0.5 * (q1 + (1-q2))
ilinx   = alpha * user_lterm[3] + (1 - alpha) * 0.5 * (q1 + q2)

In [123]:
user_sterm = user_lterm.copy()
user_sterm.loc["アゴン(競争を伴う遊び)"] = agon
user_sterm.loc["アレア(運やかけを伴う遊び)"] = alea
user_sterm.loc["ミミクリ(真似・模倣を伴う遊び)"] = mimicry
user_sterm.loc["イリンクス(目眩やスリルを伴う遊び)"] = ilinx

In [124]:
user_lterm

アゴン(競争を伴う遊び)          0.02
アレア(運やかけを伴う遊び)        0.14
ミミクリ(真似・模倣を伴う遊び)      0.08
イリンクス(目眩やスリルを伴う遊び)    0.30
dtype: float64

In [125]:
user_sterm

アゴン(競争を伴う遊び)          0.1400
アレア(運やかけを伴う遊び)        0.1675
ミミクリ(真似・模倣を伴う遊び)      0.1850
イリンクス(目眩やスリルを伴う遊び)    0.4125
dtype: float64

In [126]:
result = np.linalg.norm(data - user_sterm, axis=1)
result

array([0.69931216, 0.29922817, 0.33990808, 0.39056049, 0.40563222,
       0.65348106, 0.80500776, 0.34210744, 0.99601079, 0.64578441,
       0.88489406, 0.7389435 , 0.325634  , 0.91462424, 0.69320812,
       0.39501582, 0.69500899, 0.85383693, 0.38345469, 0.41837483,
       0.4806636 , 0.45501374, 0.72044257, 0.77203465, 0.3564232 ,
       0.30746951, 0.42371866, 0.23460072, 0.30746951, 0.34718511,
       0.30746951, 0.30746951, 0.30746951, 1.06350247, 0.95762075,
       0.42371866, 0.37287733, 0.62452982, 0.65843565, 0.57055894,
       0.30502049, 0.47700891, 0.51384579, 0.3332229 , 0.59836235,
       0.55949754, 0.42371866, 0.50203337, 0.48170271, 0.62212338,
       0.89779591])

In [127]:
rank = np.argsort(result)
rank

array([27,  1, 40, 25, 28, 30, 32, 31, 12, 43,  2,  7, 29, 24, 36, 18,  3,
       15,  4, 19, 35, 26, 46, 21, 41, 20, 48, 47, 42, 45, 39, 44, 49, 37,
        9,  5, 38, 14, 16,  0, 22, 11, 23,  6, 17, 10, 50, 13, 34,  8, 33])

In [128]:
print("おすすめ度No.1 >> {}".format(data.iloc[rank[0]].name))
print("おすすめ度No.2 >> {}".format(data.iloc[rank[1]].name))
print("おすすめ度No.3 >> {}".format(data.iloc[rank[2]].name))
print("おすすめ度No.4 >> {}".format(data.iloc[rank[3]].name))
print("おすすめ度No.5 >> {}".format(data.iloc[rank[4]].name))

おすすめ度No.1 >> 日帰りツアー
おすすめ度No.2 >> BBQ
おすすめ度No.3 >> 博物館・美術館
おすすめ度No.4 >> 日帰り温泉
おすすめ度No.5 >> 食べ歩き
